## Coroutine

In [1]:
def consumer():
    r = ''
    while True:
        n = yield r
        if not n:
            return
        print(' '*40 + f'[CONSUMER] Consuming {n}')
        r = '200 OK'

def produce(c):
    c.send(None)
    n = 0
    while n < 5:
        n = n + 1
        print(f'[PRODUCER] Producing {n}')
        r = c.send(n)
        print(f'[PRODUCER] Consumer return: {r}')
    c.close()

c = consumer()
produce(c)

[PRODUCER] Producing 1
                                        [CONSUMER] Consuming 1
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 2
                                        [CONSUMER] Consuming 2
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 3
                                        [CONSUMER] Consuming 3
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 4
                                        [CONSUMER] Consuming 4
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 5
                                        [CONSUMER] Consuming 5
[PRODUCER] Consumer return: 200 OK


注意到`consumer`函数是一个`generator`，把一个`consumer`传入`produce`后：

1. 首先调用`c.send(None)`启动生成器；
2. 然后，一旦生产了东西，通过`c.send(n)`切换到`consumer`执行；
3. `consumer`通过`yield`拿到消息，处理，又通过`yield`把结果传回；
4. `produce`拿到`consumer`处理的结果，继续生产下一条消息；
5. `produce`决定不生产了，通过`c.close()`关闭`consumer`，整个过程结束。

整个流程无锁，由一个线程执行，`produce`和`consumer`协作完成任务，所以称为“协程”，而非线程的抢占式多任务。

最后套用Donald Knuth的一句话总结协程的特点：

“子程序就是协程的一种特例。”

In [2]:
import asyncio

@asyncio.coroutine
def hello():
    print("Hello world!")
    # 异步调用asyncio.sleep(1):
    r = yield from asyncio.sleep(1)
    print("Hello again!")

# 获取EventLoop:
loop = asyncio.new_event_loop()
# 执行coroutine
loop.run_until_complete(hello())
loop.close()

Hello world!
Hello again!


自从 python 3.5 开始，python 提供了一些语法糖
```
async = @asyncio.coroutine
await = yield from
```

In [3]:
async def hello():
    print("Hello world!")
    # 异步调用asyncio.sleep(1):
    r = await asyncio.sleep(1)
    print("Hello again!")

    
loop = asyncio.new_event_loop()
loop.run_until_complete(hello())
loop.close()

Hello world!
Hello again!


In [4]:
# import time
# async def consumer(n):
#     print(' '*40 + f'[CONSUMER] Consuming {n}')
#     await asyncio.sleep(1)
#     print(' '*40 + f'[CONSUMER] Consumed {n}')
#     return '200 OK'


# async def produce():
#     n = 0
#     while n < 5:
#         n += 1
#         print(f'[PRODUCER] Producing {n}')
#         r = await consumer(n)
#         print(f'[PRODUCER] Consumer return: {r}')


# loop = asyncio.new_event_loop()
# loop.run_until_complete(produce())
# loop.close()

In [5]:
async def spin(msg):
    while True:
        await asyncio.sleep(0.5)
        print(f'spin {msg}')
    
    
async def slow_function():
    await asyncio.sleep(3)
    return 42

async def supervisor():
    spinner = asyncio.ensure_future(spin('thinging'))
    
    print('spinner object:', spinner)
    result = await slow_function()
    spinner.cancel
    return result 

def main():
    loop = asyncio.new_event_loop()
    result = loop.run_until_complete(supervisor())
    loop.close()
    print('result:',result)

main()

spinner object: <Task pending coro=<spin() running at <ipython-input-5-3ecfe550c7f2>:1>>
spin thinging
spin thinging
spin thinging
spin thinging
spin thinging
result: 42
